In [1]:
import sys
import numpy as np
sys.path.insert(0, '../')
from kwave.data import Vector
from kwave.utils.kwave_array import kWaveArray
from kwave.kgrid import kWaveGrid
from kwave.kmedium import kWaveMedium
from kwave.ksource import kSource
from kwave.ksensor import kSensor
from kwave.utils.data import scale_SI
from kwave.utils.mapgen import make_disc
from kwave.utils.signals import create_cw_signals
from kwave.utils.filters import extract_amp_phase
from kwave.kspaceFirstOrder2D import kspaceFirstOrder2D
from kwave.options import SimulationOptions, SimulationExecutionOptions

In [2]:
Nx = 64      # number of grid points in the x (row) direction

x = 1e-3     # size of the domain in the x direction [m]

dx = x / Nx  # grid point spacing in the x direction [m]

medium = kWaveMedium(sound_speed=1500)

# size of the initial pressure distribution
source_radius = 2              # [grid points]

# distance between the centre of the source and the sensor
source_sensor_distance = 10    # [grid points]

# time array
dt = 2e-9                      # [s]
t_end = 300e-9                 # [s]

# create the k-space grid
kgrid = kWaveGrid([Nx, Nx], [dx,dx])

# create the time array using an integer number of points per period
Nt: int = int(np.round(t_end / dt))
kgrid.setTime(Nt, dt)

# create instance of a sensor
sensor = kSensor()

# set sensor mask: the mask says at which points data should be recorded
sensor.mask = np.zeros((Nx,Nx), dtype=bool)

# define a single sensor point
sensor.mask[Nx // 2 + source_sensor_distance, Nx // 2] = 1

# set the record type: record the pressure waveform
sensor.record = ['p']

p0 = np.zeros((Nx, Nx))
p0[Nx//2 - source_radius: Nx//2 + source_radius] = 1.0

# make a source object
source = kSource()
source.p0 = make_disc(Vector([Nx, Nx]), Vector([Nx//2, Nx//2]), source_radius, plot_disc=False)

simulation_options = SimulationOptions(
    data_cast='single')

execution_options = SimulationExecutionOptions(
    is_gpu_simulation=True,
    delete_data=False,
    verbose_level=2)

path_to_kwave: C:\Users\dsinden\Documents\GitHub\k-wave-python\kwave
self.binary_path: C:\Users\dsinden\Documents\GitHub\k-wave-python\kwave\bin\windows\kspaceFirstOrder-CUDA.exe


In [3]:
# run the simulation
sensor_data_2D = kspaceFirstOrder2D(
    medium=medium,
    kgrid=kgrid,
    source=source,
    sensor=sensor,
    simulation_options=simulation_options,
    execution_options=execution_options)

Running k-Wave simulation...
  start time: 21-Sep-2023-20-29-15
  dt: 2ns, t_end: 298ns, time steps: 150
  input grid size: 64 by 64 grid points (1.0 by 1.0)
  maximum supported frequency: 48MHz
  smoothing p0 distribution...


AttributeError: 'kWaveSimulation' object has no attribute 'sensor_x'

In [ ]:
print(dir(sensor_data_2D), sensor_data_2D)
t_sc, t_scale, t_prefix, _ = scale_SI(t_end)
import matplotlib.pyplot as plt
_, ax1 = plt.subplots()
ax1.plot(kgrid.t_array * t_scale, sensor_data_2D / np.max(np.abs(sensor_data_2D)), 'r-')
ax1.set(xlabel= f"Time [{t_prefix}s]", ylabel='Recorded Pressure [au]')
ax1.grid(True)
plt.show()